把4个用于并行训练的模型文件，合并为一个单一的模型文件，让它可以更容易在CPU和单GPU上运行

vocab.txt 的字典大小 = 26240
词向量dim为4096

word_embeds.weight 的总大小应该是 6560 * 4, 4096

word_embeds.weight, encoder.word_embeds.weight, decoder.word_embeds.weight 这三者是同一个东西

lm_head.weight 的总大小应该是 6560 * 4, 4096，它应该是最后生成生成的词向量

encoder.blocks.0.ff.layer_norm.weight 这些东西4个文件里的都一样，存一个就好

In [1]:
import re

import torch
import numpy as np
import tensorflow as tf
from collections import OrderedDict

In [3]:
root = '../32000/'

In [4]:
print('load 0')
m0 = torch.load(root + 'mp_rank_00_model_states.pt', map_location='cpu')
print('load 1')
m1 = torch.load(root + 'mp_rank_01_model_states.pt', map_location='cpu')
print('load 2')
m2 = torch.load(root + 'mp_rank_02_model_states.pt', map_location='cpu')
print('load 3')
m3 = torch.load(root + 'mp_rank_03_model_states.pt', map_location='cpu')
print('loaded')

load 0
load 1
load 2
load 3
loaded


In [5]:
def find_weight(model, name):
    for n, w in model['module'].items():
        if name == n:
            return w, list(w.shape)

def combine(n, dim=0):
    return torch.cat([
        find_weight(x, n)[0]
        for x in (m0, m1, m2, m3)
    ], dim=dim)

In [6]:
# find_weight(m0, 'decoder.blocks.13.cross_attn.cross_attn.dense.weight')

In [7]:
new_state_dict = OrderedDict()

for n, w in m0['module'].items():
    if '.blocks.' in n:
        if 'encoder' in n:
            dim_0 = [
                'ff.dense_relu_dense.wi_0.weight',
                'ff.dense_relu_dense.wi_1.weight',
            ]
            dim_1 = [
                'self_attn.self_attn.relative_attention_bias.weight',
                'self_attn.self_attn.dense.weight',
                'ff.dense_relu_dense.wo.weight'
            ]
            dim_no = [
                'self_attn.layer_norm.weight',
                'ff.layer_norm.weight'
            ]
            clean_name = re.sub(r'^\d+', '', n[15:])[1:]
            
            if clean_name == 'self_attn.self_attn.project.weight':
                t0 = find_weight(m0, n)[0]
                t1 = find_weight(m1, n)[0]
                t2 = find_weight(m2, n)[0]
                t3 = find_weight(m3, n)[0]

                a0 = t0[:1024, :]
                b0 = t0[1024:2048, :]
                c0 = t0[2048:, :]

                a1 = t1[:1024, :]
                b1 = t1[1024:2048, :]
                c1 = t1[2048:, :]

                a2 = t2[:1024, :]
                b2 = t2[1024:2048, :]
                c2 = t2[2048:, :]

                a3 = t3[:1024, :]
                b3 = t3[1024:2048, :]
                c3 = t3[2048:, :]

                new_state_dict[n] = torch.cat([
                    a0, a1, a2, a3,
                    b0, b1, b2, b3,
                    c0, c1, c2, c3
                ], dim=0)
            elif clean_name in dim_0:
                new_state_dict[n] = combine(n, dim=0)
            elif clean_name in dim_1:
                new_state_dict[n] = combine(n, dim=1)
            elif clean_name in dim_no:
                new_state_dict[n] = find_weight(m0, n)[0]
            else:
                raise Exception('encoder what?')
        elif 'decoder' in n:
            dim_0 = [
                'cross_attn.cross_attn.project_q.weight',
                'ff.dense_relu_dense.wi_0.weight',
                'ff.dense_relu_dense.wi_1.weight',
            ]
            dim_1 = [
                'self_attn.self_attn.dense.weight',
                'self_attn.self_attn.relative_attention_bias.weight',
                'cross_attn.cross_attn.dense.weight',
                'ff.dense_relu_dense.wo.weight',
            ]
            dim_no = [
                'self_attn.layer_norm.weight',
                'cross_attn.layer_norm.weight',
                'ff.layer_norm.weight'
            ]
            clean_name = re.sub(r'^\d+', '', n[15:])[1:]

            if clean_name == 'self_attn.self_attn.project.weight':

                t0 = find_weight(m0, n)[0]
                t1 = find_weight(m1, n)[0]
                t2 = find_weight(m2, n)[0]
                t3 = find_weight(m3, n)[0]

                a0 = t0[:1024, :]
                b0 = t0[1024:2048, :]
                c0 = t0[2048:, :]

                a1 = t1[:1024, :]
                b1 = t1[1024:2048, :]
                c1 = t1[2048:, :]

                a2 = t2[:1024, :]
                b2 = t2[1024:2048, :]
                c2 = t2[2048:, :]

                a3 = t3[:1024, :]
                b3 = t3[1024:2048, :]
                c3 = t3[2048:, :]

                new_state_dict[n] = torch.cat([
                    a0, a1, a2, a3,
                    b0, b1, b2, b3,
                    c0, c1, c2, c3
                ], dim=0)
            elif clean_name == 'cross_attn.cross_attn.project_kv.weight':
                t0 = find_weight(m0, n)[0]
                t1 = find_weight(m1, n)[0]
                t2 = find_weight(m2, n)[0]
                t3 = find_weight(m3, n)[0]

                a0 = t0[:1024, :]
                b0 = t0[1024:, :]

                a1 = t1[:1024, :]
                b1 = t1[1024:, :]

                a2 = t2[:1024, :]
                b2 = t2[1024:, :]

                a3 = t3[:1024, :]
                b3 = t3[1024:, :]

                new_state_dict[n] = torch.cat([
                    a0, a1, a2, a3,
                    b0, b1, b2, b3,
                ])
            elif clean_name in dim_0:
                new_state_dict[n] = combine(n, dim=0)
            elif clean_name in dim_1:
                new_state_dict[n] = combine(n, dim=1)
            elif clean_name in dim_no:
                new_state_dict[n] = find_weight(m0, n)[0]
            else:
                raise Exception('decoder what?')
    else:
        dim_0 = [
            'word_embeds.weight',
            'lm_head.weight',
            'encoder.word_embeds.weight',
            'decoder.word_embeds.weight',
        ]
        dim_1 = [
        ]
        dim_no = [
            'encoder.final_layernorm.weight',
            'decoder.final_layernorm.weight',
        ]
        if n in dim_0:
            new_state_dict[n] = combine(n, dim=0)
        elif n in dim_1:
            new_state_dict[n] = combine(n, dim=1)
        elif n in dim_no:
            new_state_dict[n] = find_weight(m0, n)[0]
        else:
            raise Exception('other what?')

In [8]:
len(new_state_dict)

440

In [9]:
for k, v in new_state_dict.items():
    print(k, v.shape)

word_embeds.weight torch.Size([51968, 4096])
lm_head.weight torch.Size([51968, 4096])
encoder.word_embeds.weight torch.Size([51968, 4096])
encoder.final_layernorm.weight torch.Size([4096])
encoder.blocks.0.self_attn.self_attn.project.weight torch.Size([12288, 4096])
encoder.blocks.0.self_attn.self_attn.relative_attention_bias.weight torch.Size([32, 64])
encoder.blocks.0.self_attn.self_attn.dense.weight torch.Size([4096, 4096])
encoder.blocks.0.self_attn.layer_norm.weight torch.Size([4096])
encoder.blocks.0.ff.dense_relu_dense.wi_0.weight torch.Size([10240, 4096])
encoder.blocks.0.ff.dense_relu_dense.wi_1.weight torch.Size([10240, 4096])
encoder.blocks.0.ff.dense_relu_dense.wo.weight torch.Size([4096, 10240])
encoder.blocks.0.ff.layer_norm.weight torch.Size([4096])
encoder.blocks.1.self_attn.self_attn.project.weight torch.Size([12288, 4096])
encoder.blocks.1.self_attn.self_attn.dense.weight torch.Size([4096, 4096])
encoder.blocks.1.self_attn.layer_norm.weight torch.Size([4096])
encoder.

In [10]:
torch.save(new_state_dict, '../converted.zip')

In [11]:
# !ls -lh . | grep converted